In [3]:
!mkdir dataset
!curl -L -o dataset/energy_data.csv "https://raw.githubusercontent.com/yixiangD/AccurateBG/main/time-gan/data/energy_data.csv" && ls -l dataset/energy_data.csv
!curl -L -o dataset/stock_data.csv "https://raw.githubusercontent.com/yixiangD/AccurateBG/refs/heads/main/time-gan/data/stock_data.csv" && ls -l dataset/stock_data.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 4299k  100 4299k    0     0  7620k      0 --:--:-- --:--:-- --:--:-- 7663k
'ls' is not recognized as an internal or external command,
operable program or batch file.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  222k    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  222k  100  222k    0     0   773k      0 --:--:-- --:--:-- --:--:--  779k
'ls' is not recognized as an internal or external command,
operable program or batch file.
